In [38]:
import numpy as np
import pandas as pd
import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

In [2]:
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
    y = np.load(f)
f.close()

In [4]:
X = X[['OverallQual', 
       'GrLivArea',
       'GarageCars',
       'GarageArea',
       'TotalBsmtSF',
       '1stFlrSF',
       'FullBath',
       'LotShape_rank']]
X

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,LotShape_rank
0,7,1710,2,548,856,856,2,1
1,6,1262,2,460,1262,1262,2,1
2,7,1786,2,608,920,920,2,2
3,7,1717,3,642,756,961,1,2
4,8,2198,3,836,1145,1145,2,2
...,...,...,...,...,...,...,...,...
1455,6,1647,2,460,953,953,2,1
1456,6,2073,2,500,1542,2073,2,1
1457,7,2340,1,252,1152,1188,2,1
1458,5,1078,1,240,1078,1078,1,1


In [5]:
X.shape

(1460, 8)

In [7]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y =  y.reshape(-1, 1)
y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [11]:
scaled_X.shape, scaled_y.shape

((1460, 8), (1460, 1))

In [13]:
# Training
from gc import callbacks
from pickletools import optimize


model = keras.Sequential(
    [
        keras.Input(shape=scaled_X.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)

model.compile(loss='mse', optimizer='adam')

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

model.fit(scaled_X, scaled_y,
          batch_size=2, epochs=150,
          callbacks=[early_stopping_callback],
          validation_split=0.05)

Epoch 1/150


2022-08-21 17:41:11.262245: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-21 17:41:11.463524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


694/694 [==============================] - 6s 7ms/step - loss: 0.0054 - val_loss: 0.0023
Epoch 2/150
 12/694 [..............................] - ETA: 3s - loss: 0.0037

2022-08-21 17:41:16.814581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


694/694 [==============================] - 4s 5ms/step - loss: 0.0032 - val_loss: 0.0016
Epoch 3/150
694/694 [==============================] - 3s 5ms/step - loss: 0.0032 - val_loss: 0.0050
Epoch 4/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0031 - val_loss: 0.0050
Epoch 5/150
694/694 [==============================] - 3s 5ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 6/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 7/150
694/694 [==============================] - 4s 6ms/step - loss: 0.0028 - val_loss: 0.0013
Epoch 8/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0026 - val_loss: 0.0013
Epoch 9/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 10/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 11/150
694/694 [==============================] - 4s 5ms/step - loss: 0.0025 - val_loss: 0.0013


In [15]:
pred = model.predict(scaled_X[:5])
pred = y_min_max_scaler.inverse_transform(pred)

1/1 [==============================] - 0s 178ms/step


2022-08-21 17:48:39.520793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [16]:
pred

array([[189676.16],
       [164333.66],
       [216422.73],
       [204116.88],
       [292569.38]], dtype=float32)

In [17]:
y[:5]

array([[208500],
       [181500],
       [223500],
       [140000],
       [250000]])

In [25]:
model.save('./tmp/model')

INFO:tensorflow:Assets written to: ./tmp/model/assets


In [40]:
loaded_model = tf.keras.models.load_model('./tmp/model')

In [41]:
pred = loaded_model.predict(scaled_X[:5])
pred = y_min_max_scaler.inverse_transform(pred)

1/1 [==============================] - 0s 38ms/step


2022-08-21 18:06:20.676705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [42]:
pred

array([[189676.16],
       [164333.66],
       [216422.73],
       [204116.88],
       [292569.38]], dtype=float32)

In [44]:
joblib.dump(x_min_max_scaler, './tmp/x_min_max_scaler.save')
joblib.dump(y_min_max_scaler, './tmp/y_min_max_scaler.save')

['./tmp/y_min_max_scaler.save']